# __*MULTILAYER PERCEPTRON*__

## __*IMPORT LIBRARIES*__

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [26]:
data = pd.read_csv('cleaned_data.csv')
data = data.dropna(subset=['marathon_time_in_minutes']) # NOTE: dropping NaN in marathon times 
X = data[['marathon_time_in_minutes', 'height', 'age', 'HRmax', 'days', 'FFM']]
y = data[['vo2max', 'weight', 'weeklyKM']]
y.head()

,vo2max,weight,weeklyKM
0,59.7,70.7,65.0
1,46.7,71.7,110.0
2,62.0,66.2,90.0
3,61.9,67.8,65.0
4,50.2,68.3,12.5


In [27]:
# NOTE split is 70-10-20
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)

In [28]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = x_scaler.fit_transform(X_train)
X_val_scaled = x_scaler.transform(X_val)
X_test_scaled = x_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train)
y_val_scaled = y_scaler.transform(y_val)
y_test_scaled = y_scaler.transform(y_test)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

## __*DEFINE MODEL*__

In [ ]:
class MLP(nn.Module):
    def __init__(self,  num_units=32, dropout=0.3):
        super().__init__()
        self.model = nn.Sequential(
            # NOTE : This is the input layer  
            nn.Linear(6, num_units),
            nn.ReLU(),
            nn.Dropout(dropout),
            # NOTE : this is the hidden layer
            nn.Linear(num_units, num_units // 2),
            nn.ReLU(),
            # NOTE: this is the output layer
            nn.Linear(num_units // 2, 3)
        )

    def forward(self, x):
        return self.model(x)

## __*CREATE PIPELINE - wrapper for skorch*__

In [ ]:
# NOTE : Wrap model into this wrapper to adapt to skorch
regressor = NeuralNetRegressor(
    module=MLP,
    max_epochs=100,
    lr=0.01,
    optimizer=torch.optim.Adam,
    criterion=nn.MSELoss,
    iterator_train__shuffle=True,
    verbose=0  
)


## __*GRID SEARCH*__

Skorch gives the same support as in scikit-learn  

In [45]:
param_grid = {
    'lr': [0.001, 0.01, 0.1],
    'max_epochs': [100, 200],
    'module__num_units': [32, 64, 128, 256],
    'module__dropout': [0.0, 0.1, 0.2, 0.3],  # Make sure to use `module__dropout`
}

X_train_scaled = X_train_scaled.astype('float32')
y_train_scaled = y_train_scaled.astype('float32')

gs = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
gs.fit(X_train_scaled, y_train_scaled)

GridSearchCV(cv=5,
             estimator=NeuralNetRegressor(_params_to_validate={'iterator_train__shuffle'}, batch_size=128, callbacks=None, compile=False, dataset=<class 'skorch.dataset.Dataset'>, device='cpu', iterator_train=<class 'torch.utils.data.dataloader.DataLoader'>, iterator_train__shuffle=True, iterator_valid=<class 'torch.utils.data.dataloader.DataLoader'>, lr=...ss 'torch.optim.adam.Adam'>, predict_nonlinearity='auto', torch_load_kwargs=None, train_split=<skorch.dataset.ValidSplit object at 0x7f4c90115c60>, use_caching='auto', verbose=0, warm_start=False),
             param_grid={'lr': [0.001, 0.01, 0.1], 'max_epochs': [100, 200],
                         'module__dropout': [0.0, 0.1, 0.2, 0.3],
                         'module__num_units': [32, 64, 128, 256]},
             scoring='neg_mean_squared_error')

In [46]:
print("Best score:", -gs.best_score_)
print("Best params:", gs.best_params_)

Best score: 0.4583205819129944
Best params: {'lr': 0.001, 'max_epochs': 200, 'module__dropout': 0.3, 'module__num_units': 32}


## __*TRAIN MODEL*__

In [49]:
model = MLP()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training with validation
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

# Final Evaluation on Test Set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_outputs_unscaled = y_scaler.inverse_transform(test_outputs.numpy())
    print("\nSample predictions on test set:\n", test_outputs_unscaled[:5])


Epoch [10/200] | Train Loss: 1.0596 | Val Loss: 1.2458
Epoch [20/200] | Train Loss: 0.9971 | Val Loss: 1.1781
Epoch [30/200] | Train Loss: 0.9516 | Val Loss: 1.1253
Epoch [40/200] | Train Loss: 0.9207 | Val Loss: 1.0762
Epoch [50/200] | Train Loss: 0.8814 | Val Loss: 1.0217
Epoch [60/200] | Train Loss: 0.7832 | Val Loss: 0.9502
Epoch [70/200] | Train Loss: 0.7183 | Val Loss: 0.8582
Epoch [80/200] | Train Loss: 0.6569 | Val Loss: 0.7504
Epoch [90/200] | Train Loss: 0.5855 | Val Loss: 0.6336
Epoch [100/200] | Train Loss: 0.5328 | Val Loss: 0.5300
Epoch [110/200] | Train Loss: 0.4920 | Val Loss: 0.4495
Epoch [120/200] | Train Loss: 0.4630 | Val Loss: 0.4011
Epoch [130/200] | Train Loss: 0.4393 | Val Loss: 0.3772
Epoch [140/200] | Train Loss: 0.4284 | Val Loss: 0.3645
Epoch [150/200] | Train Loss: 0.4182 | Val Loss: 0.3531
Epoch [160/200] | Train Loss: 0.3871 | Val Loss: 0.3490
Epoch [170/200] | Train Loss: 0.3696 | Val Loss: 0.3460
Epoch [180/200] | Train Loss: 0.3765 | Val Loss: 0.3404
E

In [50]:

# Unscale predictions and ground truth
predictions = y_scaler.inverse_transform(test_outputs.numpy())
ground_truth = y_scaler.inverse_transform(y_test_tensor.numpy())

# Compute R² score for each output and overall
r2 = r2_score(ground_truth, predictions, multioutput='raw_values')  # individual R²
r2_mean = r2_score(ground_truth, predictions, multioutput='uniform_average')  # mean R²

print("\nR² scores per target [vo2max, weight, weeklyKM]:", r2)
print("Mean R² score across all targets:", r2_mean)


R² scores per target [vo2max, weight, weeklyKM]: [0.45491743 0.76947975 0.46048272]
Mean R² score across all targets: 0.5616266131401062
